In [0]:
from google.colab import drive
drive.mount('gdrive/')

Mounted at gdrive/


In [0]:
#define parsing function
import re
import pandas as pd

def parse_doc(doc, allkeys):

    keys = ['Message-ID']+re.findall('\n([\w\-]+):', doc[:doc.find('\n\n')])
    keys = pd.Series(keys).drop_duplicates().tolist()

    values = []
    for a, k in enumerate(keys):
        k = k+':'
        try:
            values.append(doc[doc.find(k)+len(k):doc.find(keys[a+1])].strip())
        except:
            values.append(doc[doc.find(k)+len(k):doc.find('\n\n')].strip())
    
    d = dict(zip(keys+['Body'],values+[doc[doc.find('\n\n'):].strip()]))
    k_to_remove = set(d.keys()) - set(allkeys)
    k_to_add = set(allkeys) - set(d.keys())
    
    for k in k_to_remove:
        d.pop(k)
    for k in k_to_add:
        d[k] = ''

    keys = [k[:-1] for k in keys]
    return d

def find_allkeys(docs):
    allkeys = [re.findall('\n([\w\-]+):', doc[:doc.find('\n\n')]) for doc in docs]
    allkeys = sum(allkeys,[])
    allkeys = set(allkeys)
    allkeys.add('Message-ID')
    allkeys.add('Body')
    return allkeys

In [0]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


download email.csv from here: https://www.kaggle.com/wcukierski/enron-email-dataset

make sure to put it in the same directory as where NLU-email-title folder is at.

In [0]:
#read the corpus
df = pd.read_csv('gdrive/My Drive/NLU project/raw_data/emails.csv')

In [0]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [0]:
'''sampling 1000 entries for now to cut down runtime'''
docs = df.message.sample(1000)

In [0]:
#test
allkeys = find_allkeys(docs)
allkeys

{'Bcc',
 'Body',
 'Cc',
 'Content-Transfer-Encoding',
 'Content-Type',
 'Date',
 'From',
 'Message-ID',
 'Mime-Version',
 'Subject',
 'To',
 'X-FileName',
 'X-Folder',
 'X-From',
 'X-Origin',
 'X-To',
 'X-bcc',
 'X-cc'}

In [0]:
for i in docs:
    print(i)
    print('\n--------Parsed-------\n')
    print(parse_doc(i,allkeys))
    break

Message-ID: <26460695.1075854410150.JavaMail.evans@thyme>
Date: Tue, 13 Jun 2000 06:45:00 -0700 (PDT)
From: darron.giron@enron.com
To: imelda.frayre@enron.com
Subject: Re: New ENA - IM Mid Business Units
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Darron C Giron
X-To: Imelda Frayre
X-cc: 
X-bcc: 
X-Folder: \Darron_Giron_Jun2001\Notes Folders\Sent
X-Origin: Giron-D
X-FileName: dgiron.nsf

ENA-IM Mid Central South
ANR SW
PEPL
OGT (ONG)
Reliant
Williams
MRT
Transok
Enogex

ENA-IM Mid Central North
NNG
Northern Border
NGPL

DG    3-9573



Imelda Frayre

06/13/2000 01:37 PM
To: Darron C Giron/HOU/ECT@ECT
cc: George Smith/HOU/ECT@ECT, Kam Keiser/HOU/ECT@ECT 
Subject: New ENA - IM Mid Business Units

Darron, do you know which pipes these two new business units will be 
trading?  I need to make sure that all necessary pipes are mapped to these 
business units in order for traders to be able to book deals on them.  Let me 
know if there 

In [0]:
#parsing data
parsed = list()
for doc in docs:
    d = parse_doc(doc, allkeys)
    parsed.append(d)

In [0]:
parse = pd.DataFrame(parsed)

In [0]:
#save data
parse.to_csv('gdrive/My Drive/NLU project/raw_data/parse_1000.csv')